# Clasificación de galaxias

El objetivo de esta tarea es hacer una red neuronal que pueda diferenciar entre tres tipos de galaxia: espiral, irregular y elíptica. Se va a hacer por medio de una red convolucional (para disminuír el número de parámetros de la red.

El ejercicio está basado en el [tutorial](https://www.tensorflow.org/tutorials/layers) de tensorflow para capas en una red convolutiva.

Las fotos de estas galaxias las necesitan sacar del folder zip que se llame 'source' (basta con darle click derecho y luego descomprimir, en realidad.

Primero lo primero, cargar las bibliotecas necesarias. Recuerden, si una les aparece que no está instalada, basta con que utilicen el ambiente virtual para instalar cosas. En particular, tensorflow y PIL (cuyo paquete se llama Pillow, es decir, `pip install pillow`).

In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
from matplotlib.pyplot import imshow
from PIL import Image
import os

ModuleNotFoundError: No module named 'tensorflow'

Ahora vamos a cargar en RAM todas las fotos. Para ello primero cargamos los nombres y sus tipos (lo único que nos interesa). Me aprovecho de ello para de una vez guardar en un data frame sólo las columnas de interés. No sólo eso pero de una vez vamos a convertir las etiquetas en las tres posibilidades de galaxia que nos interesa clasificar: Elíptica ()

In [ ]:
data = #Cargar los datos aquí porfaaaa
numToGalaxy = {0: 'Eliptica', 1: 'Espiral', 2: 'Irregular'}

Podemos ver aquí los primeros 5 renglones de este dataframe

In [ ]:
data.head()

Esperemos tener una distribución masomenos equitativa de cada galaxia. Usando `groupby` podremos verificar esto

In [ ]:
data.groupby("Type").count()

Pues... Resulta que no. En fin, A la mejor el problema fue dar un rango tan grande a las espirales

In [ ]:
data.groupby("HStage").count().Type

Si pues, un poco más homogeneo... Tenemos aún que cargar 82MB de imágenes en RAM. Esto puede ser un grave problema.

In [ ]:
# From the dataframe, we make a list of images
images = []

images = data.apply(lambda row: Image.open(os.path.join('source',row.PhotoName+'.png')).convert('L'), axis = 1)
shape = [images[0].height, images[0].width]
# To normalized numpy array
images = np.array(
    list(map(lambda pic: np.reshape(np.array(pic), [1, pic.width * pic.height])[0] , images)), dtype = 'float32'
) / 255
print(images.shape)

El siguiente bloque es la parte clave: esriba la estructura de redes convolucionales interna para procesar cada imagen. La última capa y la primer capa ya están hechas. De por medio puede utilizar _pooling_, _flattening_, redes de conexión completa o convoluciones. Como sugerencia: use 2 filtros de 8x8 (convoluciones), luego haga pooling de 3x3 al resultado, luego puede aplanar el resultado y hacer alguna red pequeña de conexión completa con probabilidad de dejar caer algunas aristas.

In [ ]:
def cnn_model_fn(features, labels, mode):
    """Model function for CNN."""
    
    # Input Layer, regresamos las imagenes a su dimensión
    input_layer = tf.reshape(features["x"], [-1, shape[0], shape[1], 1])

    
    
    
    ### AQUÍ TODAS LAS CONEXIONES INTERMEDIAS ###
    
    
    
    
    # Logits Layer, ternary classification
    logits = tf.layers.dense(inputs=lastLayer, units=3)

    predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

Ahora ponemos el estimador. Como nota, el folder tmp se debe borrar para cada nuevo modelo que se haga, o el entrenamiento continuará desde el último valor de los pesos de cada arista.

In [ ]:
# Se crea ahora el estimador, en un folder llamado tmp
cs_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn, model_dir="./tmp/cs_model")

Aquí se hace el entrenamiento, vaya por un número de repeticiones coherente. El argumento batch_size es el número de fotos que se usarán para calcular el gradiente para cada paso.

In [ ]:
# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": #Información en x},
    y = #Información en y,
    batch_size=1,
    num_epochs=None,
    shuffle=True)
cs_classifier.train(input_fn=train_input_fn,steps=1)

Ahora, para hacer una pequeña prueba y que puedan ver su red funcionando, el siguiente bloque de código carga una imagen de manera aleatoria, y les dice qué tipo de galaxia es (la que su red debería predecir)

In [ ]:
idx = np.random.randint(0,600)
testImg = Image.open(os.path.join('source',data.PhotoName[idx]+'.png')).convert('L')
imshow(testImg)
print('La clasificación de la galaxia es ' + numToGalaxy[data.NType[idx]])

Este bloque que viene adelante, ejecuta la red para hacer una sola predicción. `pred` es una lista que contiene las probabilidades de cada clase

In [ ]:
X = np.reshape(np.array(testImg,dtype = 'float32'), [1, testImg.width * testImg.height])[0] / 255

predict_input_fn = tf.estimator.inputs.numpy_input_fn(
    x = {'x': np.reshape(X, [1, -1]).astype('float32')},
    shuffle = False
)
pred = list(cs_classifier.predict(input_fn = predict_input_fn))

print('Predicciones: {}'.format(str(pred)))
print('Debieramos tener una galaxia ' + numToGalaxy[pred[0]['classes']])

# Paso último y más difícil

Busque, google o stackoverflow, cómo hacer una validación cruazada para su red neuronal. Usted tiene 600 fotos, separe 40 de cada grupo para tener 80% de la información para entrenamiento y 20% para hacer una prueba.

Lo puede hacer de manera manual separando la información en 5 bloques, y entrenando con las 5 posibles permutaciones.